<a href="https://colab.research.google.com/github/valerio-unifei/chemotherapy-safety/blob/main/RNCC_Ampolas_Frascos_Seringas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificação de Frascos, Ampolas e Seringas

## Dataset

### Baixando imagens do Google Drive

A pasta de imagens já é subdivida nas classes para classificação:
- Ampoule
- BottleA
- BottleB
- Syringe

In [ ]:
import gdown

# Arquivo no Drive com dataset de imagens completo
# https://drive.google.com/file/d/1n8LaNDE9GT1LYFG1jGcwbs3OMb4EMZj7/

# baixando dataset no colab
fullDS = 'https://drive.google.com/uc?id=1n8LaNDE9GT1LYFG1jGcwbs3OMb4EMZj7'
gdown.download(fullDS, 'FullDataset.zip', quiet=False)

#descompactando e apagando arquivo compactado
!unzip -qq FullDataset.zip -d fullds
!rm FullDataset.zip

Downloading...
From: https://drive.google.com/uc?id=1n8LaNDE9GT1LYFG1jGcwbs3OMb4EMZj7
To: /content/FullDataset.zip
286MB [00:03, 73.7MB/s]


### Carregando Dataset no Formato Tensor para treinamento e validação

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

directory = '/content/fullds'

# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory
image_size = (128, 128)
batch_size = 32
cores = 'rgb' 
#cores = 'grayscale'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=directory,
    validation_split=0.2, # 80%
    subset='training',
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
    color_mode = cores,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=directory,
    validation_split=0.2, # 20%
    subset='validation',
    seed=42,
    image_size=image_size,
    batch_size=batch_size,
    color_mode = cores,
)

class_names = train_ds.class_names

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

class_names

Found 1423 files belonging to 4 classes.
Using 1139 files for training.
Found 1423 files belonging to 4 classes.
Using 284 files for validation.


['Ampoule', 'BottleA', 'BottleB', 'Syringe']

## Modelo da rede neural por convolução para classificação

Esta rede tem como entrada uma imagem (largura, altura, cores) e saída o número de classes para classificar:
- Ampoule (neuronio de saída 0)
- BottleA (neuronio de saída 1)
- BottleB (neuronio de saída 2)
- Syringe (neuronio de saída 3)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(class_names)))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 50176)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

## Treinamento com 20 iterações

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_ds, epochs=20, validation_data=val_ds,)

Epoch 1/20
36/36 [==============================] - 34s 99ms/step - loss: 20.0389 - accuracy: 0.7191 - val_loss: 0.5538 - val_accuracy: 0.9049
Epoch 2/20
36/36 [==============================] - 9s 217ms/step - loss: 0.3826 - accuracy: 0.8982 - val_loss: 0.4662 - val_accuracy: 0.8979
Epoch 3/20
36/36 [==============================] - 9s 221ms/step - loss: 0.2046 - accuracy: 0.9385 - val_loss: 0.2097 - val_accuracy: 0.9261
Epoch 4/20
36/36 [==============================] - 10s 223ms/step - loss: 0.2606 - accuracy: 0.9368 - val_loss: 0.3371 - val_accuracy: 0.9225
Epoch 5/20
36/36 [==============================] - 9s 221ms/step - loss: 0.1853 - accuracy: 0.9526 - val_loss: 0.4014 - val_accuracy: 0.9261
Epoch 6/20
36/36 [==============================] - 9s 215ms/step - loss: 0.0892 - accuracy: 0.9781 - val_loss: 0.2014 - val_accuracy: 0.9331
Epoch 7/20
36/36 [==============================] - 9s 216ms/step - loss: 0.0688 - accuracy: 0.9816 - val_loss: 0.2265 - val_accuracy: 0.9437
Epoc

# Validação de Modelo

Matriz de confusão

In [ ]:
import numpy as np

predictions = np.array([])
labels =  np.array([])
#dados de validação
for x, y in val_ds:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis=-1)])
  labels = np.concatenate([labels,y.numpy()])
#dados de treinamento
for x, y in train_ds:
  predictions = np.concatenate([predictions, np.argmax(model.predict(x), axis=-1)])
  labels = np.concatenate([labels,y.numpy()])

#matriz de confusão do conjunto completo
tf.math.confusion_matrix(labels=labels, predictions=predictions).numpy()

array([[  34,    2,    1,    4],
       [   0,   77,    0,    1],
       [   0,    1,   66,    4],
       [   0,    0,    1, 1232]], dtype=int32)